# Numbers and arithmetic in Julia

*adapted from David Sander's "Hands-on Julia" notebook [1. Numbers, variables and basic functions.ipynb]( https://github.com/dpsanders/hands_on_julia/blob/master/notebooks/Session%20I%20-%20Basic%20Julia/1.%20Numbers%2C%20variables%20and%20basic%20functions.ipynb)*

This notebook will teach you about Julia's built-in numerical types and familiarize you with Julia notebooks. After completing the notebook, you should know 

  * some of Julia's types for integers, rationals, real numbers, and complex numbers
  * how to do arithmetic operations on those types (+ - * / ^)
  * some of quirks of finite-precision computational arithmetic (round-off and overflow, in particular)
  * how to construct values of different numerical types and how to convert between them
  * how to do high-precision computations
  * how to enter Unicode variable names


## Integers, $\mathbb{Z}$

**Problem 1:** Type different types of integers and try different operations with them. What does `3/4` do? What about `3**4`? (Use `shift-return` to execute the current cell in IJulia. Use `control-m b` to open a new cell below the current.)


In [2]:
3 + 4

7

**Problem 2:** Calculate powers of 10. Keep going.... What happens, eventually?

In [14]:
10^10

10000000000

Good. Keep going until you get to...

In [16]:
10^18

1000000000000000000

In [17]:
10^19

-8446744073709551616

Whoa! 10^19 should be 1 with 19 zeroes. It is definitely not a negative number. What happened is that 10^n eventually got too big to fit in a 64-bit integer type, and the calculation [overflowed](https://en.wikipedia.org/wiki/Integer_overflow).

Types are crucial in Julia (although we often do not need to mention them explicitly).
The function `typeof` tells us what type an object is:

In [18]:
typeof(10)

Int64

Note that functions in Julia take argument lists inside parentheses, as above.

**Problem 3:** We can do arbitrary-precision arithmetic using the `big` function. Calculate some powers of 10 with arbitrary precision. What happens if you do `big(10^20)`? Why?

In [19]:
big(10)^20

100000000000000000000

In [20]:
big(10^20)

7766279631452241920

`big(10)^20` works because it converts to `BigInt` before taking the power. `big(10^20)` fails because it computes `10^20` with Int64 arithmetic (which overflows) before converting to a `BigInt`.

**Problem 4:** Calculate $2^{2^{2^{2^2}}}$ with normal arithmetic and in arbitrary precision.

In [21]:
2^2^2^2^2

0

In [25]:
big(2)^2^2^2^2

2003529930406846464979072351560255750447825475569751419265016973710894059556311453089506130880933348101038234342907263181822949382118812668869506364761547029165041871916351587966347219442930927982084309104855990570159318959639524863372367203002916969592156108764948889254090805911457037675208500206671563702366126359747144807111774815880914135742720967190151836282560618091458852699826141425030123391108273603843767876449043205960379124490905707560314035076162562476031863793126484703743782954975613770981604614413308692118102485959152380195331030292162800160568670105651646750568038741529463842244845292537361442533614373729088303794601274724958414864915930647252015155693922628180691650796381064132275307267143998158508811292628901134237782705567421080070065283963322155077831214288551675554073345107213112427399562982719769150054883905223804357045848197956393157853510018992000024141963706813559840464039472194016069517690156119726982337890017641517190051133466306898140219383481435426387306539552

Whoa, that's BIG!

**Problem 5:** Use the `string` function to convert the previous result (called `ans`) into a string. How could we calculate the number of digits in the number? [Hint: try typing the first few letters of a function and use `<TAB>` to find possible completions.]

In [26]:
length(string(ans))

19729

## Rationals, $\mathbb{Q}$

There is a built-in rational type in the Julia standard library. Rationals are constructed using the `//` operator.

**Problem 6:** Calculate $\frac{3}{4} + \frac{5}{6}$.

In [27]:
3//4 + 5//6

19//12

**Problem 7:** Calculate powers of $\frac{3}{4}$. What happens? What is the solution? What is the type of the resulting objects?

In [33]:
(3//4)^10

59049//1048576

In [32]:
typeof(ans)

Rational{Int64}

In [39]:
(3//4)^32

LoadError: [91mOverflowError()[39m

## Reals, $\mathbb{R}$

Real numbers are approximated by floating-point numbers. Julia has several different floating-point types available whose names start with `Float`, and also the arbitrary-precision type 'BigFloat'.

**Problem 8:**  Use tab completion to find the available floating-point types. 

In [41]:
Float16, Float32, Float64, BigFloat

(Float16, Float32, Float64, BigFloat)

**Problem 9:** Perform some type conversions like `Float32(1)` or `convert(Float32, 1)` to see how Julia prints the different floating-point numbers. If you type back in exactly what Julia prints out, you'll enter the given number as a *floating point literal* of a specific type. Try this with a variety of floating-point types. 

In [42]:
Float32(1)

1.0f0

In [43]:
x = 1.0f0

1.0f0

In [44]:
Float16(1)

Float16(1.0)

**Problem 10:** What happens when we calculate $2.3 \times 4.6$? Why?

In [45]:
2.3 * 4.6

10.579999999999998

It should be exactly 10.58, but it's not! The issue is that neither 2.3 nor 4.6 can be represented exactly as finite-precision binary numbers. Their exact binary representation are infinitely repeating. So they're approximated with truncated binary expansions. When those numbers are multiplied, again with finite-precision arithmetic, the answer is not exactly 10.58. 

**Problem 11:** What type is the answer? (This tells us the default floating-point type.)

In [46]:
typeof(ans)

Float64

**Problem 12:** Use the `bits` function to examine the internal representation of `2.3`.

In [47]:
bits(2.3)

"0100000000000010011001100110011001100110011001100110011001100110"

There you see the repeating pattern in the binary expansion of 2.3.

"Scientific notation" may be used for large and small numbers, e.g. `1.3e100`.

Julia has built-in support for arbitrary-precision floating-point numbers:

**Problem 13:** What happens if we do `big(0.1)`? What is the resulting type? What is strange about the answer? Why did that happen?

In [48]:
big(0.1)

1.000000000000000055511151231257827021181583404541015625000000000000000000000000e-01

0.1 is a `Float64` literal, which has about sixteen digits of precision. So 0.1 gets rounded to sixteen digits and then converted to a `BigFloat`. So the trailing digits are of course wrong. 

**Problem 14:** Can you think of way to get a `BigFloat` value that is 0.1 to (nearly) the last digit? Hint: do some arithmetic on full-precision `BigFloat` numbers.

In [49]:
1/big(10)

1.000000000000000000000000000000000000000000000000000000000000000000000000000002e-01

**Problem 15:** Two other ways to enter full-precision `BigFloat` numbers are `BigFloat("0.1")` and `big"0.1"`. Give these a try. 

In [53]:
BigFloat("0.1")

1.000000000000000000000000000000000000000000000000000000000000000000000000000002e-01

In [54]:
big"0.1"

1.000000000000000000000000000000000000000000000000000000000000000000000000000002e-01

**Problem 16:** Change to a larger precision using `setprecision`. (Note that help is available on functions using `?NAME`, where `NAME` is the name of the function.) Then create an even more accurate value for 0.1.  

In [58]:
setprecision(512)

512

In [59]:
big"0.1"

1.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002e-01

**Problem 17:** Compute a 1024-bit value of $\pi$. How many digits is it accurate to? 

In [62]:
setprecision(1024)
BigFloat(π)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724586997

In [63]:
length(string(ans))

311

**Problem 18:** What is $\sin(\pi)$ using the 1024-bit value of $\pi$?

In [65]:
sin(BigFloat(π))

9.133833352024599180125447717198210546898671161281024272611229443223053781604030291859166872648904775362024742973537644471991734000035832845360935620115603852146149714433229210713707343080592505078336200427722640480861209277927521424947883394313007151900266906161947352678704040638571335512803354533018167991488e-309

## Complex numbers, $\mathbb{C}$

Julia also has built-in support for complex numbers. The imaginary unit, $i = \sqrt{-1}$, is denoted by `im` in Julia. 

**Problem 19:** Check that Julia knows that $i^2 = -1$.

In [66]:
im^2

-1 + 0im

**Problem 20:** How are complex numbers written? What is the type of a complex number formed with integers? With reals? With rationals? This starts to tell us how Julia's types work.

In [67]:
4+5im

4 + 5im

In [68]:
typeof(ans)

Complex{Int64}

In [69]:
4.0 + 5.0im

4.0 + 5.0im

In [70]:
typeof(ans)

Complex{Float64}

In [75]:
3//4 + 5im//7

3//4 + 5//7*im

In [76]:
typeof(ans)

Complex{Rational{Int64}}

**Problem 21:** Guess the name of the function to calculate the complex conjugate. Use it to calculate the absolute value of $3 + 4i$. Check it against the absolute value function.

In [77]:
conj(3+4im)

3 - 4im

In [78]:
x = 3+4im
abs(x), sqrt(x*conj(x))

(5.0, 5.0 + 0.0im)

# Variables 

Variables in Julia are defined directly using the assignment operator. No type declaration is necessary.

In [79]:
x = 3

3

**Problem 22:** `x` has an automatically assigned type. What is its type? Can it change type?

In [80]:
typeof(x)

Int64

In [81]:
x = 4.5

4.5

In [82]:
typeof(x)

Float64

Yes, `x` can change type.

**Problem 23:** Declare a constant `xx` using `const`? What happens if you try to change its value?

In [83]:
const xx = 4

4

In [84]:
xx = 5

5

In [85]:
xx = 5.0

LoadError: [91minvalid redefinition of constant xx[39m

In Julia, `const` applies to the **type** of the variable. Changing the value of a `const` gives a warning, but changing the type of a const gives an error.

**Problem 24:** Variable names can contain any Unicode character (although the first letter of the variable name is restricted to be letter-like). Julia allows many useful Unicode characters to be typed using LaTeX notation: type  `\alpha<TAB>`.  You can type `\alp` and then type `<TAB>` to see possible completions.

In [ ]:
α = 3; ℵ = 10 # note: statements on the same line can be separated with semicolons

**Problem 25:** See what Greek letters you can generate in Julia, small and capital. Try some other characters if you can. 

In [86]:
α, β, γ, δ, ϵ, ζ, η, θ # and so on!
Α, Β, Γ, Δ, Ε, Ζ, Η, Θ # capital greek letters

LoadError: [91mUndefVarError: α not defined[39m

## Review of numerical types

**Problem 27:** How many numerical types are there in Julia? How many can you name? (Besides the ones we've covered, there are also unsigned integers like `UInt32` and `Uint64`.) 

In [ ]:
# The concrete numeric types

Bool
Int8, Int16, Int32, Int64, Int128, BigInt
UInt8, UInt16, UInt32, UInt64, UInt128, UBigInt
Float16, Float32, Float64, BigFloat
Irrational

In [92]:
# Complex and Rational numbers are formed from those, for example

Int8(3)//Int8(7)

3//7

In [93]:
typeof(ans)

Rational{Int8}

In [96]:
setprecision(128)
big"7.4" + big"1.3"*im

7.400000000000000000000000000000000000005 + 1.299999999999999999999999999999999999998im

In [98]:
big(3)//4 - big(7)//2 * im

3//4 - 7//2*im

In [99]:
typeof(ans)

Complex{Rational{BigInt}}

With this combinatoric explosion of combinations of Integer, Rational, Float, and Complex types,  there are A LOT of numeric types in Julia. You could count them, but I'll just guess there are fifty or so numeric types in the base Julia libraries. You can explore Julia's type system with the `subtype` and `supertype` function and the `<:` subtype operator.

In [90]:
subtypes(Number)

2-element Array{Union{DataType, UnionAll},1}:
 Complex
 Real   

In [100]:
subtypes(Integer)

4-element Array{Union{DataType, UnionAll},1}:
 BigInt  
 Bool    
 Signed  
 Unsigned

In [108]:
supertype(Bool)

Integer

In [102]:
Int64 <: Signed # is Int64 a subtype of the signed integers?

true

In [104]:
Integer <: Real # are the integers a subtype of the reals?

true

In [107]:
Int32 <: Int64  # are Int32s a subtype of Int64s? No! They're a subset, but not a subtype

false

See also ￼
  * [Integers and floating-point numbers, Julia documentation](https://docs.julialang.org/en/stable/manual/integers-and-floating-point-numbers/#Integers-and-Floating-Point-Numbers-1)
  * [Complex and rational numbers, Julia documentation](https://docs.julialang.org/en/stable/manual/complex-and-rational-numbers/#Complex-and-Rational-Numbers-1)
  * [Julia numeric types, Wikibooks](https://en.wikibooks.org/wiki/Introducing_Julia/Types)